# Exploratory Data Analysis and Preprocessing

In [4]:
import pandas as pd

In [5]:
# Load master CSV file
food_df = pd.read_csv('./data/combined_food_data.csv')

# Inspect DataFrame to determine the best way to combine them
print("Menu Data:", food_df.shape)

Menu Data: (213, 62)
